In [33]:
#split train and test data
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

cwd = os.getcwd()
df = pd.read_csv(cwd + "/data_csv/preprocessing_data.csv")

train, test = train_test_split(df, test_size=0.1)


In [34]:
train = pd.DataFrame(train, columns=["image", "emotion"])
train.to_csv(cwd + "/data_csv/train.csv")

train.groupby('emotion').count()

,image
emotion,
ANGER,71
DISGUST,6
HAPPINESS,2126
NEUTRAL,2008
SADNESS,31
SURPRISE,78


In [35]:
test = pd.DataFrame(test, columns=["image", "emotion"])
test.to_csv(cwd + "/data_csv/test.csv")

test.groupby('emotion').count()

,image
emotion,
ANGER,7
HAPPINESS,233
NEUTRAL,228
SADNESS,5
SURPRISE,8


In [36]:
'''
import cv2,matplotlib.pyplot as plt,dlib,imutils
from imutils import face_utils

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../input/dlib-68/shape_predictor_68_face_landmarks.dat")

image=plt.imread("../input/recognizing-faces-in-the-wild/train/F0002/MID1/P00009_face3.jpg")
# image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 1)

for rect in rects:
    pred=predictor(gray,rect)
    fig, ax1 = plt.subplots()

    ax1.imshow(image)
    ax1.scatter(face_utils.shape_to_np(pred)[:,0],face_utils.shape_to_np(pred)[:,1])
    
# del predictor
'''

'\nimport cv2,matplotlib.pyplot as plt,dlib,imutils\nfrom imutils import face_utils\n\ndetector = dlib.get_frontal_face_detector()\npredictor = dlib.shape_predictor("../input/dlib-68/shape_predictor_68_face_landmarks.dat")\n\nimage=plt.imread("../input/recognizing-faces-in-the-wild/train/F0002/MID1/P00009_face3.jpg")\n# image = imutils.resize(image, width=500)\ngray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\nrects = detector(gray, 1)\n\nfor rect in rects:\n    pred=predictor(gray,rect)\n    fig, ax1 = plt.subplots()\n\n    ax1.imshow(image)\n    ax1.scatter(face_utils.shape_to_np(pred)[:,0],face_utils.shape_to_np(pred)[:,1])\n    \n# del predictor\n'

In [37]:
'''
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

data = pd.read_csv(cwd + "/train_data.csv")

#independent columns
X = data.iloc[:,0:20]

#target column
y = data.iloc[:,-1]  

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns, dfscores],axis=1)

#naming the dataframe columns
featureScores.columns = ['Specs','Score']

#print 10 best features
print(featureScores.nlargest(10,'Score'))
'''

'\nfrom sklearn.feature_selection import SelectKBest\nfrom sklearn.feature_selection import chi2\n\ndata = pd.read_csv(cwd + "/train_data.csv")\n\n#independent columns\nX = data.iloc[:,0:20]\n\n#target column\ny = data.iloc[:,-1]  \n\n#apply SelectKBest class to extract top 10 best features\nbestfeatures = SelectKBest(score_func=chi2, k=10)\nfit = bestfeatures.fit(X,y)\ndfscores = pd.DataFrame(fit.scores_)\ndfcolumns = pd.DataFrame(X.columns)\n\n#concat two dataframes for better visualization \nfeatureScores = pd.concat([dfcolumns, dfscores],axis=1)\n\n#naming the dataframe columns\nfeatureScores.columns = [\'Specs\',\'Score\']\n\n#print 10 best features\nprint(featureScores.nlargest(10,\'Score\'))\n'

In [38]:
'''
from sklearn.preprocessing import StandardScaler

label = train.head(1000)
data = train.head(1000)
data.shape

std = StandardScaler().fit_transform(data)
std.shape
'''

'\nfrom sklearn.preprocessing import StandardScaler\n\nlabel = train.head(1000)\ndata = train.head(1000)\ndata.shape\n\nstd = StandardScaler().fit_transform(data)\nstd.shape\n'

In [39]:
df = train
dictionary = {'emotion':{'HAPPINESS':0,'NEUTRAL':1, 'SURPRISE':2, 'ANGER':3, 'SADNESS':4, 'DISGUST':5}}
df.replace(dictionary, inplace = True)

In [40]:
df

,image,emotion
4463,Tom_OBrien_0001.jpg,0
3594,Paul_Sarbanes_0003.jpg,1
3103,Mary_Carey_0004.jpg,5
4033,Saddam_Hussein_0012.jpg,0
2084,Jane_Walker_Wood_0001.jpg,0
...,...,...
3166,Meryl_Streep_0004.jpg,0
4324,Taku_Yamasaki_0001.jpg,1
1084,Diane_Lane_0001.jpg,0
2719,Kofi_Annan_0014.jpg,1


In [52]:
'''
    Jawline = 0–16 กราม
    Eyebrows = 17–26 คิ้ว
    Nose = 27–35 จมูก
    Left eye = 36–41 ตาซ้าย
    Right eye = 42–47 ตาขวา
    Mouth = 48–67 ปาก
'''
stat = [
    [0, 16 + 1], # jawline
    [17, 26 + 1], # eyebrow
    [27, 35 + 1], # nose
    [36, 41 + 1], # left_eye
    [42, 47 + 1], # right_eye
    [48, 67 + 1] # mount
]
def get_avg_shape(shape, face_position = 0):
    (start, end) = stat[face_position]
    summation = [shape[index] for index in range(start, end)]
    x = [float(shape[index][0]) for index in range(start, end)]
    y = [float(shape[index][1]) for index in range(start, end)]
    return {
        "x": np.mean(x),
        "y" : np.mean(y)
    }


In [54]:
import cv2, glob, random, math, numpy as np, dlib, itertools
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from imutils import face_utils

# emotion list
# {'HAPPINESS':0,'NEUTRAL':1, 'SURPRISE':2, 'ANGER':3, 'SADNESS':4, 'DISGUST':5}
emotion_type = ['HAPPINESS', 'NEUTRAL', 'SURPRISE', 'ANGER', 'SADNESS', 'DISGUST']
emotions = [0, 1, 2, 3, 4, 5, 6] 
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(cwd + "/predictor/shape_predictor_68_face_landmarks.dat")

error = 0
arr_face_type = [
    [],
    [],
    [],
    [],
    [],
    []
]
data_frame = {
    "image" : [],
    "name" : [],
    "emotion" : [],
    "jaw_x" : [],
    "jaw_y" : [],
    "eye_brows_x": [],
    "eye_brows_y": [],
    "nose_x" : [],
    "nose_y": [],
    "eye_left_x": [],
    "eye_left_y" : [],
    "eye_right_x" : [],
    "eye_right_y" : [],
    "mount_x" : [],
    "mount_y" : []
}
# run each image in train
for idx, row in df.iterrows():
    print(row)
    imagePath = cwd + "/cleaned_images/" + row.image
    image = cv2.imread(imagePath)

    rects = detector(image, 0)

    if len(rects) == 0:
        error += 1
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(image, rect)
        shape = face_utils.shape_to_np(shape)
    
        # loop over the (x, y)-coordinates for the facial landmarks
        # and draw them on the image
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)

        # store x and y coordinates in two lists
        # xlist = []
        # ylist = []
        # for i in range(0,len(shape)):
        #     xlist.append(float(shape[i][0]))
        #     ylist.append(float(shape[i][1]))
        emotion = int(row.emotion)
        mount = [[float(shape[index][0]), float(shape[index][1])] for index in range(48, 67 + 1)]
        mount_x_arr = [float(shape[index][0]) for index in range(48, 67 + 1)]
        mount_y_arr = [float(shape[index][1]) for index in range(48, 67 + 1)]
        # eyebrows = [x for x in range(17, 26 + 1)]
        # nose = [x for x in range(27, 35 + 1)]
        # left_eye = [x for x in range(36, 41 + 1)]
        # right_eye = [x for x in range(42, 47 + 1)]
        # mount = [x for x in range(48, 67 + 1)]
        print("Emotion : ", emotion_type[emotion])
        data = {
            "jawline" : get_avg_shape(shape, 0),
            "eye_brows" : get_avg_shape(shape, 1),
            "nose" : get_avg_shape(shape, 2),
            "left_eye" : get_avg_shape(shape, 3),
            "right_eye" : get_avg_shape(shape, 4),
            "mount" : get_avg_shape(shape, 5),
        }
        print(data)
        arr_face_type[emotion].append(
            data
        )

        '''
            Jawline = 0–16 กราม
            Eyebrows = 17–26 คิ้ว
            Nose = 27–35 จมูก
            Left eye = 36–41 ตาซ้าย
            Right eye = 42–47 ตาขวา
            Mouth = 48–67 ปาก
        '''
        # get mean of both axes to determine centre of gravity
        # xmean = np.mean(xlist) 
        # ymean = np.mean(ylist)

        # print(xmean, ymean)
        
        '''
        # get distance between each point and the central point
        xcentral = [(x - xmean) for x in xlist] 
        ycentral = [(y - ymean) for y in ylist]
        '''
            
    # show the output image with the face detections + facial landmarks
    cv2.imshow("Output", image)
    k = cv2.waitKey(5) & 0xFF
    if k == 68:
        break

print(error)

image      Tom_OBrien_0001.jpg
emotion                      0
Name: 4463, dtype: object
Emotion :  HAPPINESS
{'jawline': {'x': 192.76470588235293, 'y': 264.94117647058823}, 'eye_brows': {'x': 162.1, 'y': 103.8}, 'nose': {'x': 159.44444444444446, 'y': 202.55555555555554}, 'left_eye': {'x': 99.5, 'y': 131.66666666666666}, 'right_eye': {'x': 232.5, 'y': 126.5}, 'mount': {'x': 174.65, 'y': 285.7}}
image      Paul_Sarbanes_0003.jpg
emotion                         1
Name: 3594, dtype: object
Emotion :  NEUTRAL
{'jawline': {'x': 165.2941176470588, 'y': 271.70588235294116}, 'eye_brows': {'x': 164.3, 'y': 96.4}, 'nose': {'x': 176.77777777777777, 'y': 209.55555555555554}, 'left_eye': {'x': 105.16666666666667, 'y': 148.0}, 'right_eye': {'x': 230.0, 'y': 141.0}, 'mount': {'x': 178.7, 'y': 291.55}}
image      Mary_Carey_0004.jpg
emotion                      5
Name: 3103, dtype: object
Emotion :  DISGUST
{'jawline': {'x': 192.7058823529412, 'y': 273.88235294117646}, 'eye_brows': {'x': 171.9, 'y': 10

KeyboardInterrupt: 